## DOWNLOADING SCRIPT

This notebook is meant for use on the CDSE platform.

This notebook requires no input data.

The output of the notebook will be GeoJSON files with ground-level data on ambient particulate matter in the atmosphere.

See: CAMS Air Qu

_CAUTION:_ CAMS downloads have a tendency to crash, in which case the user must babysit the downloading process a bit.

*TODO:* rework the downloading process.

In [44]:
#!pip install xarray  numpy  cdsapi  
# Python Standard Libraries
import os
import datetime as dt
import zipfile

# Data Manipulation Libraries
import numpy as np
import xarray as xr

# libraries
import geopandas
from shapely.geometry import Point

# Climate Data Store API for retrieving climate data
import cdsapi

In [49]:
# key to atmosphere monitoring platform
URL =  'https://ads.atmosphere.copernicus.eu/api'
KEY = # get from platform

In [47]:
# define working directory
%cd /home/jovyan/mystorage/testdir

/home/jovyan/mystorage/testdir


In [90]:
# decide later if these variables are needed
variables = ["co", "ectot", "no", "no2", "o3", "pm10", "sia"]
c = cdsapi.Client(url=URL, key=KEY)

# swe fi bbox
# redefine bounding box if desired
xmin = 11.00
ymin = 55.30
xmax = 32.00
ymax = 70.17

# define any desired years
#years = ["2019", "2020", "2021", "2022", "2023"]
years = ["2022", "2023"]
# define any desired months
months = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

# API request automatically generated by the CAMS service
dataset = "cams-europe-air-quality-reanalyses"

for yyyy in years:
    for mm in months:
        if ((yyyy == "2022") & ((mm == "02"))): # for some reason request fails for this month???
            continue
        if ((yyyy != "2022") & ((mm != "01") )):
            request = {
                "variable": [ # TODO: remove unnecessary variables
                    "carbon_monoxide",
                    "nitrogen_dioxide",
                    "nitrogen_monoxide",
                    "ozone",
                    "secondary_inorganic_aerosol",
                    "total_elementary_carbon",
                    "particulate_matter_10um",
                    "sulphur_dioxide"
                ],
                "model": ["lotos"], # EUROS-LOTOS model reanalysis by ECMWF
                "level": ["0"], # ground-level
                "type": ["interim_reanalysis"],
                "year": [yyyy],
                "month": [
                    mm
                ],
                 "data_format": "nc5", # faster download
                "download_format": "archived" # faster download
            }
            
            c.retrieve(dataset, request, target="download.nc")
            with zipfile.ZipFile("download.nc", "r") as zip_ref:
                zip_ref.extractall("download"+yyyy)

            # extract SO2 first
            fname = "download"+yyyy+"/cams.eaq.ira.KNMa.so2.l0."+yyyy+"-"+mm+".nc"
            xr_so2 = xr.open_dataset(fname, engine="netcdf4")
            # slice sweden-finland
            data_swefi = xr_so2.sel(lon=slice(xmin, xmax), lat=slice(ymin, ymax))
        
    
        for var in variables:
    
            fname = "download"+yyyy+"/cams.eaq.ira.KNMa."+var+".l0."+yyyy+"-"+mm+".nc"
            xrds = xr.open_dataset(fname, engine="netcdf4")
            data_swefi[var] = xrds.sel(lon=slice(xmin, xmax), lat=slice(ymin, ymax))[var]
    
        agg_data = data_swefi.mean(dim="time")
        #agg_data.to_netcdf("./data/demo2-y"+yyyy+"m"+mm+".nc")

        # collection cell: I am too lazy to clean this notebook
        
        # dataframe-ify and turn into geojson
        agg_data_df = agg_data.to_dataframe()
        agg_data_df = agg_data_df.reset_index()
        geom = [Point(x,y) for x, y in zip(agg_data_df['lon'], agg_data_df['lat'])]
        geodf_agg = geopandas.GeoDataFrame(agg_data_df, geometry=geom)
        geodf_agg.head()
        geodf_agg.to_file("./data/"+yyyy+"-"+mm+"demo.geojson", driver="GeoJSON")
        

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/mystorage/testdir/download2022/cams.eaq.ira.KNMa.co.l0.2022-04.nc'

### DEPRECATED CODE CELLS BELOW

## DO NOT RUN, ONLY READ OR USE AS MODEL

In [52]:
with zipfile.ZipFile("download", "r") as zip_ref:
        zip_ref.extractall("download"+yyyy)

In [94]:
# deprecated
"""
request = {
    "variable": [
        "carbon_monoxide",
        "nitrogen_dioxide",
        "nitrogen_monoxide",
        "ozone",
        "secondary_inorganic_aerosol",
        "total_elementary_carbon",
        "particulate_matter_10um",
        "sulphur_dioxide"
    ],
    "model": ["lotos"],
    "level": ["0"],
    "type": ["interim_reanalysis"],
    "year": ["2022"],
    "month": [
        "02"
    ],
     "data_format": "nc5",
    "download_format": "archived"
}
"""
#c.retrieve(dataset, request, target="download.nc")

2025-03-09 19:26:32,045 WARNING [2025-03-09T19:26:32.024057] You are using a deprecated API endpoint. If you are using cdsapi, please upgrade to the latest version.
2025-03-09 19:26:32,051 INFO Request ID is bf7c070b-589c-47b0-9957-c11dd52b7b8b
2025-03-09 19:26:32,234 INFO status has been updated to accepted
2025-03-09 19:26:44,719 WARNING Download format not supported for this dataset. Defaulting to zip.
2025-03-09 19:26:44,724 INFO status has been updated to running
2025-03-09 19:30:49,776 INFO status has been updated to successful


c67140dbece64e2df15f05486e4d6511.zip:   0%|          | 0.00/5.89G [00:00<?, ?B/s]

'download.nc'

In [57]:
# downloading keeps crashing
# try processing manually
variables

['co', 'dust', 'ectot', 'no', 'no2', 'o3', 'pm10', 'sia']

In [67]:
fname = "download"+yyyy+"/cams.eaq.ira.KNMa.so2.l0.2022-03.nc"
xr_so2 = xr.open_dataset(fname, engine="netcdf4")

In [68]:
data_swefi = xr_so2.sel(lon=slice(xmin, xmax), lat=slice(ymin, ymax))

In [69]:
variables = ["co", "ectot", "no", "no2", "o3", "pm10", "sia"]

#yyyy = "2023"
yyyy = "2022"

for var in variables:

    fname = "download2022/cams.eaq.ira.KNMa."+var+".l0.2022-03.nc"
    xrds = xr.open_dataset(fname, engine="netcdf4")
    data_swefi[var] = xrds.sel(lon=slice(xmin, xmax), lat=slice(ymin, ymax))[var]

agg_data = data_swefi.mean(dim="time")
agg_data.to_netcdf("./data/demo2-y"+yyyy+".nc")

In [70]:
agg_data

<xarray.Dataset> Size: 1MB
Dimensions:  (lon: 210, lat: 149)
Coordinates:
  * lon      (lon) float64 2kB 11.05 11.15 11.25 11.35 ... 31.75 31.85 31.95
  * lat      (lat) float64 1kB 55.35 55.45 55.55 55.65 ... 69.95 70.05 70.15
Data variables:
    so2      (lat, lon) float32 125kB 0.6869 0.7826 0.8963 ... 0.7103 0.7625
    co       (lat, lon) float32 125kB 161.1 167.6 170.5 ... 148.6 148.4 148.3
    ectot    (lat, lon) float32 125kB 0.4532 0.484 0.5382 ... 0.03084 0.03069
    no       (lat, lon) float32 125kB 1.16 0.6969 0.5498 ... 0.008949 0.009127
    no2      (lat, lon) float32 125kB 7.805 5.925 5.049 ... 0.2518 0.2522 0.2527
    o3       (lat, lon) float32 125kB 75.55 74.1 66.64 ... 89.33 89.27 89.26
    pm10     (lat, lon) float32 125kB 17.66 17.95 18.49 ... 9.593 9.625 9.692
    sia      (lat, lon) float32 125kB 9.835 10.08 10.24 ... 1.588 1.584 1.576

In [6]:
import zipfile

In [7]:
with zipfile.ZipFile("e19b02f99a17469232c0db6c09b6dd96.zip","r") as zip_ref:
    zip_ref.extractall("data/download")

In [4]:
# import the SO2
nc_so2 = "data/download/cams.eaq.ira.KNMa.so2.l0.2023-07.nc"

xr_so2 = xr.open_dataset(nc_so2, engine="netcdf4")

In [5]:
xr_so2

<xarray.Dataset> Size: 875MB
Dimensions:  (time: 744, lon: 700, lat: 420)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2023-07-01 ... 2023-07-31T23:00:00
  * lon      (lon) float32 3kB -24.95 -24.85 -24.75 -24.65 ... 44.75 44.85 44.95
  * lat      (lat) float32 2kB 30.05 30.15 30.25 30.35 ... 71.75 71.85 71.95
Data variables:
    so2      (time, lat, lon) float32 875MB ...
Attributes:
    Conventions:  CF-1.7
    Title:        CAMS European air quality interim reanalysis
    Provider:     COPERNICUS European air quality service
    Production:   COPERNICUS Atmosphere Monitoring Service

In [6]:
#bbox_swefin = BBox([11.020, 55.30, 32.00, 70.17], crs=CRS.WGS84).transform(CRS(3857))

In [7]:
xmin = 11.00
ymin = 55.30
xmax = 32.00
ymax = 70.17

In [8]:
xr_so2_swefi = xr_so2.sel(lon=slice(xmin, xmax), lat=slice(ymin, ymax))

In [9]:
xr_so2_swefi

<xarray.Dataset> Size: 93MB
Dimensions:  (time: 744, lon: 210, lat: 149)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2023-07-01 ... 2023-07-31T23:00:00
  * lon      (lon) float32 840B 11.05 11.15 11.25 11.35 ... 31.75 31.85 31.95
  * lat      (lat) float32 596B 55.35 55.45 55.55 55.65 ... 69.95 70.05 70.15
Data variables:
    so2      (time, lat, lon) float32 93MB ...
Attributes:
    Conventions:  CF-1.7
    Title:        CAMS European air quality interim reanalysis
    Provider:     COPERNICUS European air quality service
    Production:   COPERNICUS Atmosphere Monitoring Service

In [10]:
xr_so2_swefi.so2

<xarray.DataArray 'so2' (time: 744, lat: 149, lon: 210)> Size: 93MB
[23279760 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 6kB 2023-07-01 ... 2023-07-31T23:00:00
  * lon      (lon) float32 840B 11.05 11.15 11.25 11.35 ... 31.75 31.85 31.95
  * lat      (lat) float32 596B 55.35 55.45 55.55 55.65 ... 69.95 70.05 70.15
Attributes:
    standard_name:                 mass_concentration_of_sulfur_dioxide_in_air
    long_name:                     mass concentration of sulfur dioxide
    units:                         µg/m3
    number_of_significant_digits:  3
    source:                        mass concentration of sulfur dioxide at 0 ...

### Downloading values experiment

In [11]:
xr_so2_swefi

<xarray.Dataset> Size: 93MB
Dimensions:  (time: 744, lon: 210, lat: 149)
Coordinates:
  * time     (time) datetime64[ns] 6kB 2023-07-01 ... 2023-07-31T23:00:00
  * lon      (lon) float32 840B 11.05 11.15 11.25 11.35 ... 31.75 31.85 31.95
  * lat      (lat) float32 596B 55.35 55.45 55.55 55.65 ... 69.95 70.05 70.15
Data variables:
    so2      (time, lat, lon) float32 93MB ...
Attributes:
    Conventions:  CF-1.7
    Title:        CAMS European air quality interim reanalysis
    Provider:     COPERNICUS European air quality service
    Production:   COPERNICUS Atmosphere Monitoring Service

In [12]:
data_swefi = xr_so2_swefi

In [16]:
variables = ["co", "dust", "ectot", "no", "no2", "o3", "pm10", "sia"]

for var in variables:

    fname = "data/download/cams.eaq.ira.KNMa."+var+".l0.2023-07.nc"
    xrds = xr.open_dataset(fname, engine="netcdf4")
    data_swefi[var] = xrds[var]

In [19]:
t0 = "2023-07-01T12:00:00"
t1 = "2023-07-31T12:00:00"

In [22]:
data_swefi.sel(time=[t0, t1])

<xarray.Dataset> Size: 2MB
Dimensions:  (time: 2, lon: 210, lat: 149)
Coordinates:
  * time     (time) datetime64[ns] 16B 2023-07-01T12:00:00 2023-07-31T12:00:00
  * lon      (lon) float32 840B 11.05 11.15 11.25 11.35 ... 31.75 31.85 31.95
  * lat      (lat) float32 596B 55.35 55.45 55.55 55.65 ... 69.95 70.05 70.15
Data variables:
    so2      (time, lat, lon) float32 250kB ...
    co       (time, lat, lon) float32 250kB ...
    dust     (time, lat, lon) float32 250kB ...
    ectot    (time, lat, lon) float32 250kB ...
    no       (time, lat, lon) float32 250kB ...
    no2      (time, lat, lon) float32 250kB ...
    o3       (time, lat, lon) float32 250kB ...
    pm10     (time, lat, lon) float32 250kB ...
    sia      (time, lat, lon) float32 250kB ...
Attributes:
    Conventions:  CF-1.7
    Title:        CAMS European air quality interim reanalysis
    Provider:     COPERNICUS European air quality service
    Production:   COPERNICUS Atmosphere Monitoring Service

In [24]:
data_swefi.sel(time=[t0, t1]).to_netcdf("./data/demo1.nc")

### GeoPandas experiments

In [71]:
# collection cell: I am too lazy to clean this notebook
# libraries
import geopandas
from shapely.geometry import Point

# dataframe-ify the 
agg_data_df = agg_data.to_dataframe()
agg_data_df = agg_data_df.reset_index()
geom = [Point(x,y) for x, y in zip(agg_data_df['lon'], agg_data_df['lat'])]
geodf_agg = geopandas.GeoDataFrame(agg_data_df, geometry=geom)
geodf_agg.head()
geodf_agg.to_file("./data/0322demo.geojson", driver="GeoJSON")

In [72]:
geodf_agg

,lon,lat,so2,co,ectot,no,no2,o3,pm10,sia,geometry
0,11.05,55.35,0.686881,161.124741,0.453183,1.159885,7.804810,75.554634,17.661661,9.834899,POINT (11.05000 55.35000)
1,11.05,55.45,0.611537,158.876892,0.405424,0.589704,5.123747,78.184448,17.464296,9.846318,POINT (11.05000 55.45000)
2,11.05,55.55,0.603566,158.734665,0.397017,0.421696,4.188167,78.497643,17.309443,9.755754,POINT (11.05000 55.55000)
3,11.05,55.65,0.874059,165.914688,0.457504,0.674063,5.732825,75.593452,17.877903,9.925507,POINT (11.05000 55.65000)
4,11.05,55.75,0.692359,161.418015,0.403106,0.412463,4.057068,77.179749,16.793957,9.578770,POINT (11.05000 55.75000)
...,...,...,...,...,...,...,...,...,...,...,...
31285,31.95,69.75,1.380116,144.081696,0.038266,0.013655,0.348096,86.602531,6.369429,1.603353,POINT (31.95000 69.75000)
31286,31.95,69.85,1.297200,146.069855,0.034849,0.013400,0.327246,87.804222,7.178958,1.593327,POINT (31.95000 69.85000)
31287,31.95,69.95,1.182105,147.992767,0.033112,0.012102,0.296824,89.076843,6.792434,1.608530,POINT (31.95000 69.95000)
31288,31.95,70.05,1.007965,147.916702,0.031571,0.009904,0.269554,89.041519,9.390265,1.583653,POINT (31.95000 70.05000)


In [34]:
import geopandas
from shapely.geometry import Point

In [28]:
df_t0 = data_swefi.sel(time=t0).to_dataframe()
df_t1 = data_swefi.sel(time=t1).to_dataframe()

In [32]:
df_t0 = df_t0.reset_index()
df_t0.head()

,lon,lat,time,so2,co,dust,ectot,no,no2,o3,pm10,sia
0,11.049999,55.349998,2023-07-01 12:00:00,0.091888,132.0000,0.097046,0.086121,0.507568,2.430664,73.81250,5.386719,1.483398
1,11.049999,55.449997,2023-07-01 12:00:00,0.127136,130.5625,0.085693,0.073669,0.343140,1.892090,72.75000,5.582031,1.846680
2,11.049999,55.549999,2023-07-01 12:00:00,0.110168,128.7500,0.074860,0.061172,0.242615,1.455078,71.59375,5.443359,1.982910
3,11.049999,55.650002,2023-07-01 12:00:00,0.151306,129.2500,0.074005,0.061676,0.347046,1.778809,68.81250,3.973633,1.881348
4,11.049999,55.750000,2023-07-01 12:00:00,0.106079,128.5625,0.089691,0.057281,0.298584,1.622559,67.06250,4.089844,1.752930


In [35]:
#df_t0.reset_index()
geom = [Point(x,y) for x, y in zip(df_t0['lon'], df_t0['lat'])]
geodf_t0 = geopandas.GeoDataFrame(df_t0, geometry=geom)
geodf_t0.head()

,lon,lat,time,so2,co,dust,ectot,no,no2,o3,pm10,sia,geometry
0,11.049999,55.349998,2023-07-01 12:00:00,0.091888,132.0000,0.097046,0.086121,0.507568,2.430664,73.81250,5.386719,1.483398,POINT (11.05000 55.35000)
1,11.049999,55.449997,2023-07-01 12:00:00,0.127136,130.5625,0.085693,0.073669,0.343140,1.892090,72.75000,5.582031,1.846680,POINT (11.05000 55.45000)
2,11.049999,55.549999,2023-07-01 12:00:00,0.110168,128.7500,0.074860,0.061172,0.242615,1.455078,71.59375,5.443359,1.982910,POINT (11.05000 55.55000)
3,11.049999,55.650002,2023-07-01 12:00:00,0.151306,129.2500,0.074005,0.061676,0.347046,1.778809,68.81250,3.973633,1.881348,POINT (11.05000 55.65000)
4,11.049999,55.750000,2023-07-01 12:00:00,0.106079,128.5625,0.089691,0.057281,0.298584,1.622559,67.06250,4.089844,1.752930,POINT (11.05000 55.75000)


In [36]:
## geometry Point
df_t1 = df_t1.reset_index()
df_t1.head()
geom1 = [Point(x,y) for x, y in zip(df_t1['lon'], df_t1['lat'])]
geodf_t1 = geopandas.GeoDataFrame(df_t1, geometry=geom)
geodf_t1.head()

,lon,lat,time,so2,co,dust,ectot,no,no2,o3,pm10,sia,geometry
0,11.049999,55.349998,2023-07-31 12:00:00,0.227295,141.1875,0.193054,0.088104,0.556396,1.639160,70.84375,14.195312,1.340820,POINT (11.05000 55.35000)
1,11.049999,55.449997,2023-07-31 12:00:00,0.157532,139.4375,0.182190,0.074066,0.445190,1.381348,72.37500,12.558594,1.095215,POINT (11.05000 55.45000)
2,11.049999,55.549999,2023-07-31 12:00:00,0.105804,136.0625,0.172729,0.063293,0.488525,1.425293,71.12500,12.027344,0.895264,POINT (11.05000 55.55000)
3,11.049999,55.650002,2023-07-31 12:00:00,0.154053,136.5000,0.146240,0.052872,0.360840,1.100586,71.12500,10.777344,0.737061,POINT (11.05000 55.65000)
4,11.049999,55.750000,2023-07-31 12:00:00,0.124512,135.0000,0.118744,0.039719,0.206116,0.669922,68.46875,11.085938,0.633545,POINT (11.05000 55.75000)


In [40]:
geodf_t0.drop("time", axis=1).to_file("./data/0701demo.geojson", driver="GeoJSON")
geodf_t1.drop("time", axis=1).to_file("./data/0731demo.geojson", driver="GeoJSON")